In [1]:
import urllib.request 
from urllib.parse import quote 
from bs4 import BeautifulSoup
import json
import re
import time

In [2]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [3]:
#Source: https://codezup.com/web-scraping-word-meaning-dictionary-python-beautifulsoup/

In [4]:
import json
with open('./bigFreqList.json') as json_file:
     wordsData = json.load(json_file)

In [5]:
#generate a list of words with frequency higher than the threshold
def generateWordList(threshold): 
  temp = []
  for (word, freq) in wordsData.items():
    if (freq >= threshold):
      temp.append(word)
  return temp

In [6]:
wordList = generateWordList(40)
print(wordList[-10:])
len(wordList)

['latikivn', 'imusicvn', 'ycđt', 'greatland', 'lemmevn', 'hoclamgiau', 'hpschoolvn', 'khochiasevn', 'hakiba', 'heisvn']


141836

In [7]:
#break the word list into batches of 
batches = [wordList[i:i + 10000] for i in range(0, len(wordList), 10000)] 

In [8]:
for batch in batches:
  print(len(batch))

10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
1836


In [9]:
#The link format for vdict Viet-Eng is: https://vdict.com/m%E1%BB%99t,2,0,0.html

In [10]:
#Converting IRI to ASCII:
#https://stackoverflow.com/questions/4389572/how-to-fetch-a-non-ascii-url-with-python-urlopen

In [11]:
def findTranslation(word):
    print("Finding translation for " + word)
    url = "https://vdict.com/" + quote(word) + ",2,0,0.html" #resolving the IRI issue
    
    try: #make sure the link is working
        source = urllib.request.urlopen(url)
    except:
        print("Link broken for " + word)
        return "N/A"
    
    soup = BeautifulSoup(source, 'lxml')

    #make sure the word is in the dictionary
    if (soup.find("div", id = 'result-contents') is None):
        print("Not in the dictionary " + word)
        return "N/A"
    
    translations = {}

    #Getting the first POS tag
    firstPosTag = soup.find("div", class_ = 'phanloai')
    
    if (firstPosTag is None): #empty translation page
        print("Empty translation page " + word)
        return "N/A"
    
    curPos = firstPosTag.string.split(' \xa0')[0]

    #first translation is a tag away from the first POS tag
    curTag = firstPosTag.next_sibling.next_sibling

    #storing the number of translations a word has
    numTrans = 1

    #continue while we are either in a POS tag or a translation tag
    while (curTag != '\n'): 
        if (curTag.attrs["class"] == ["list1"]): #a translation
            temp = {} #temp dict to store each translation of a word

            #getting the raw translation
            if (curTag.find('b').string is not None): #empty translation cell
                temp['translation'] = re.split('[,;]', curTag.find('b').string)
                temp['POS'] = curPos

                #retrieving the examples of each of the translations
                try:
                    examples = curTag.findAll("ul", class_= "list2")
                    temp2 = {} #another dict to store each examples of a translation

                    numExamples = len(list(examples))
                    temp2['nums'] = numExamples

                    for j in range(0, numExamples):
                        temp2['context ' + str(j+1)] = examples[j].find(class_ = "example-original").string

                        #retrieving the usage of each examples
                        try: 
                            tempString = str(examples[j].find("li")) #extracting the usage
                            temp2['usage ' + str(j+1)] = str(examples[j].find("li").contents[-1])
                        except:
                            temp2['usage ' + str(j+1)] = "N/A"

                    temp['examples'] = temp2
                except:
                    temp['examples'] = "N/A"

                translations[numTrans] = temp #adding the translation to the dictionary

                numTrans += 1
            curTag = curTag.next_sibling                
        else: #a tag
            curPos = curTag.string.split(' \xa0')[0]
            curTag = curTag.next_sibling

    #storing the number of translations
    translations['nums'] = numTrans - 1
    return translations

In [12]:
url = "https://vdict.com/" + quote("có") + ",2,0,0.html" #resolving the IRI issue

try:
    source = urllib.request.urlopen(url)
except:
    print( "Link broken")
soup = BeautifulSoup(source, 'lxml')

In [13]:
soup.find("div", class_ = 'phanloai').next_sibling.next_sibling.next_sibling.next_sibling

'\n'

In [14]:
#from google.colab import files
start_time = time.time()

index = 3
batch = batches[index] + batches[index+1]

#for (index, batch) in enumerate(batches):
dictionary = {}

#making the dictionary for each batch
for word in batch:
    dictionary[word] = findTranslation(word)
    time.sleep(1) #wait for 1 sec between each query

Finding translation for dany
Not in the dictionary dany
Finding translation for dane
Not in the dictionary dane
Finding translation for daejeon
Not in the dictionary daejeon
Finding translation for dz
Not in the dictionary dz
Finding translation for dmm
Not in the dictionary dmm
Finding translation for dhq
Not in the dictionary dhq
Finding translation for dh04
Not in the dictionary dh04
Finding translation for d20
Not in the dictionary d20
Finding translation for d001
Not in the dictionary d001
Finding translation for crete
Not in the dictionary crete
Finding translation for crafted
Not in the dictionary crafted
Finding translation for conversion
Not in the dictionary conversion
Finding translation for condoleezza
Not in the dictionary condoleezza
Finding translation for cienco4
Not in the dictionary cienco4
Finding translation for chôl
Not in the dictionary chôl
Finding translation for chnăm
Not in the dictionary chnăm
Finding translation for chart
Not in the dictionary chart
Finding 

Finding translation for têu
Not in the dictionary têu
Finding translation for têm
Finding translation for tâm1
Not in the dictionary tâm1
Finding translation for trằng
Not in the dictionary trằng
Finding translation for trạn
Not in the dictionary trạn
Finding translation for tróng
Not in the dictionary tróng
Finding translation for triệungày
Not in the dictionary triệungày
Finding translation for transducer
Not in the dictionary transducer
Finding translation for thnh
Not in the dictionary thnh
Finding translation for thọt
Finding translation for thấtchúng
Not in the dictionary thấtchúng
Finding translation for tatami
Not in the dictionary tatami
Finding translation for soắn
Not in the dictionary soắn
Finding translation for somat
Not in the dictionary somat
Finding translation for seam
Not in the dictionary seam
Finding translation for sapiens
Not in the dictionary sapiens
Finding translation for rổn
Not in the dictionary rổn
Finding translation for roadtrip
Not in the dictionary road

Finding translation for thẻn
Not in the dictionary thẻn
Finding translation for techcrunch
Not in the dictionary techcrunch
Finding translation for tanzanite
Not in the dictionary tanzanite
Finding translation for tampak
Not in the dictionary tampak
Finding translation for taejo
Not in the dictionary taejo
Finding translation for taehoon
Not in the dictionary taehoon
Finding translation for tpvị
Not in the dictionary tpvị
Finding translation for tktech
Not in the dictionary tktech
Finding translation for tkl
Not in the dictionary tkl
Finding translation for thermotronic
Not in the dictionary thermotronic
Finding translation for tbình
Not in the dictionary tbình
Finding translation for tvan
Not in the dictionary tvan
Finding translation for superspeed
Not in the dictionary superspeed
Finding translation for sullair
Not in the dictionary sullair
Finding translation for stepan
Not in the dictionary stepan
Finding translation for statham
Not in the dictionary statham
Finding translation fo

Finding translation for kyoung
Not in the dictionary kyoung
Finding translation for kuciak
Not in the dictionary kuciak
Finding translation for kitakyushu
Not in the dictionary kitakyushu
Finding translation for kewpie
Not in the dictionary kewpie
Finding translation for karina
Not in the dictionary karina
Finding translation for kanon
Not in the dictionary kanon
Finding translation for kajin
Not in the dictionary kajin
Finding translation for k80
Not in the dictionary k80
Finding translation for k400
Not in the dictionary k400
Finding translation for jiangsu
Not in the dictionary jiangsu
Finding translation for jennings
Not in the dictionary jennings
Finding translation for jenn
Not in the dictionary jenn
Finding translation for jazzfit
Not in the dictionary jazzfit
Finding translation for jve
Not in the dictionary jve
Finding translation for json
Not in the dictionary json
Finding translation for joc
Not in the dictionary joc
Finding translation for isoflavones
Not in the dictionary 

Finding translation for bamboonet
Not in the dictionary bamboonet
Finding translation for bambino
Not in the dictionary bambino
Finding translation for balong
Not in the dictionary balong
Finding translation for backstreet
Not in the dictionary backstreet
Finding translation for bqđ
Not in the dictionary bqđ
Finding translation for bjt
Not in the dictionary bjt
Finding translation for bctn
Not in the dictionary bctn
Finding translation for aum
Not in the dictionary aum
Finding translation for around
Not in the dictionary around
Finding translation for amplified
Not in the dictionary amplified
Finding translation for amit
Not in the dictionary amit
Finding translation for altieri
Not in the dictionary altieri
Finding translation for almagro
Not in the dictionary almagro
Finding translation for allow
Not in the dictionary allow
Finding translation for allinone
Not in the dictionary allinone
Finding translation for alcott
Not in the dictionary alcott
Finding translation for aimee
Not in t

...

Finding translation for bottom
Not in the dictionary bottom
Finding translation for bosniaherzegovina
Not in the dictionary bosniaherzegovina
Finding translation for boniancol
Not in the dictionary boniancol
Finding translation for bones
Not in the dictionary bones
Finding translation for bombee
Not in the dictionary bombee
Finding translation for boheme
Not in the dictionary boheme
Finding translation for blumhouse
Not in the dictionary blumhouse
Finding translation for bluffs
Not in the dictionary bluffs
Finding translation for bluff
Not in the dictionary bluff
Finding translation for bluesea
Not in the dictionary bluesea
Finding translation for blackrock
Not in the dictionary blackrock
Finding translation for biogreen
Not in the dictionary biogreen
Finding translation for biocellulose
Not in the dictionary biocellulose
Finding translation for billion
Not in the dictionary billion
Finding translation for beria
Not in the dictionary beria
Finding translation for beppu
Not in the dicti

Finding translation for 712017nđcp
Not in the dictionary 712017nđcp
Finding translation for 700tr
Not in the dictionary 700tr
Finding translation for 70000m2
Not in the dictionary 70000m2
Finding translation for 7các
Not in the dictionary 7các
Finding translation for 6x15m
Not in the dictionary 6x15m
Finding translation for 6mg
Not in the dictionary 6mg
Finding translation for 6h50
Not in the dictionary 6h50
Finding translation for 6h40
Not in the dictionary 6h40
Finding translation for 6h14h
Not in the dictionary 6h14h
Finding translation for 68kg
Not in the dictionary 68kg
Finding translation for 650mm
Not in the dictionary 650mm
Finding translation for 650k
Not in the dictionary 650k
Finding translation for 652013nđcp
Not in the dictionary 652013nđcp
Finding translation for 64b
Not in the dictionary 64b
Finding translation for 5x12m
Not in the dictionary 5x12m
Finding translation for 5dbi
Not in the dictionary 5dbi
Finding translation for 5w30
Not in the dictionary 5w30
Finding tran

Not in the dictionary trendline
Finding translation for vietnet
Not in the dictionary vietnet
Finding translation for vnds
Not in the dictionary vnds
Finding translation for viresa
Not in the dictionary viresa
Finding translation for vif
Not in the dictionary vif
Finding translation for viasm
Not in the dictionary viasm
Finding translation for nmc
Not in the dictionary nmc
Finding translation for nasati
Not in the dictionary nasati
Finding translation for kctv
Not in the dictionary kctv
Finding translation for iffhs
Not in the dictionary iffhs
Finding translation for icr
Not in the dictionary icr
Finding translation for baseline
Not in the dictionary baseline
Finding translation for apru
Not in the dictionary apru
Finding translation for 00h30
Not in the dictionary 00h30
Finding translation for vvip
Not in the dictionary vvip
Finding translation for stranger
Not in the dictionary stranger
Finding translation for mizuiku
Not in the dictionary mizuiku
Finding translation for buon
Not in 

Not in the dictionary polyolefin
Finding translation for pijama
Not in the dictionary pijama
Finding translation for phẩmđặc
Not in the dictionary phẩmđặc
Finding translation for phẩmthiết
Not in the dictionary phẩmthiết
Finding translation for phược
Not in the dictionary phược
Finding translation for phươngđiều
Not in the dictionary phươngđiều
Finding translation for phươn
Not in the dictionary phươn
Finding translation for phútchế
Not in the dictionary phútchế
Finding translation for phíhiệu
Not in the dictionary phíhiệu
Finding translation for pháttriển
Not in the dictionary pháttriển
Finding translation for perfluorooctanoic
Not in the dictionary perfluorooctanoic
Finding translation for paranoid
Not in the dictionary paranoid
Finding translation for oxacilin
Not in the dictionary oxacilin
Finding translation for oligosaccharides
Not in the dictionary oligosaccharides
Finding translation for officinalis
Not in the dictionary officinalis
Finding translation for nướcđó
Not in the dic

KeyboardInterrupt: 

In [ ]:
#saving the batch
with open('./vdict VI-EN batch {}.json'.format("34"), 'w') as outfile:
  json.dump(dictionary, outfile)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
findTranslation("cái")